In [1]:
import scipy.io
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from d2l import torch as d2l
import argparse
import datetime
import wandb 

In [3]:
# 参数解析器
parser = argparse.ArgumentParser()
parser.add_argument('--project_name', type=str, default='DSC_CNN_demo01', help='project_name')
parser.add_argument('--batch_size', type=int, default=32, help='size of each image batch')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--optim_type', type=str, default='Adam', help='Optimizer')
parser.add_argument('--epochs', type=int, default=100, help='epochs')
parser.add_argument('--model_pth_name', type=str, default='../save/model_default.pth', help='pth_name')
# 不会解析命令行参数
args = parser.parse_args([])
# 返回一个NameSpace对象

In [10]:
def min_max_normalize(data, dim=-1):
    """
    对输入数据在指定维度上进行 Min - Max 归一化
    """
    min_vals = data
    min_vals, _ = torch.min(min_vals, dim=dim, keepdim=True)
    max_vals = data
    max_vals, _ = torch.max(max_vals, dim=dim, keepdim=True)
    
    denominator = max_vals - min_vals
    denominator[denominator == 0] = 1e-8
    
    normalized_data = (data - min_vals) / denominator
    return normalized_data

def create_input_iamge(data, length):
    num = len(data)//(length**2)
    data = data[:num*(length**2)].reshape((num, 1, length, length))
    data = torch.tensor(data, dtype=torch.float32)
    # Max-Min 归一化
    data = min_max_normalize(data, dim=-1)
    return data

def create_dataloaders(length, Data_names, Data_names_key, Labels, is_train=1, batch_size = args.batch_size):
    for i in range(len(Data_names)):
        print('正在读取：' + Data_names[i])
        mat_data = scipy.io.loadmat('../data/' + Data_names[i])
        numpy_X = mat_data[Data_names_key[i]]
        print('ndarry 形状：', numpy_X.shape)
        tensor_X = create_input_iamge(numpy_X, length)
        tensor_y = torch.full((tensor_X.shape[0],), Labels[i], dtype=torch.int64)
        print('tensor 形状：', tensor_X.shape)
        if i==0:
            X = tensor_X
            y = tensor_y
        else:
            # dim是轴而不是方向i
            X = torch.cat([X, tensor_X], dim=0)
            y = torch.cat([y, tensor_y], dim=0)
    print('模型输入的 tensor 形状为：', X.shape)
    print('对应标签的 tensor 形状为：', y.shape)
    
    dataset = TensorDataset(X, y)
    if is_train:
        train_size = int(0.7 * len(dataset))
        val_size = len(dataset) - train_size
        # 随机73划分数据集
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        train_dataloader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
            drop_last=True
        )
        val_dataloader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
            drop_last=True
        )
        return train_dataloader, val_dataloader
    
    else:
        infer_dataloader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=2,
            drop_last=True
        )
        return infer_dataloader

In [5]:
Data_names = ['normal_0', 'IR007_0', 'B007_0', 'OR007@6_0']
Data_names_key = ['X097_FE_time', 'X278_FE_time', 'X282_FE_time', 'X294_FE_time']
Labels = [0, 1, 2, 3]

In [14]:
create_dataloaders(32, Data_names, Data_names_key, Labels, is_train=0, batch_size = args.batch_size)

正在读取：normal_0
ndarry 形状： (243938, 1)
tensor 形状： torch.Size([238, 1, 32, 32])
正在读取：IR007_0
ndarry 形状： (121535, 1)
tensor 形状： torch.Size([118, 1, 32, 32])
正在读取：B007_0
ndarry 形状： (121168, 1)
tensor 形状： torch.Size([118, 1, 32, 32])
正在读取：OR007@6_0
ndarry 形状： (121168, 1)
tensor 形状： torch.Size([118, 1, 32, 32])
模型输入的 tensor 形状为： torch.Size([592, 1, 32, 32])
对应标签的 tensor 形状为： torch.Size([592])


In [12]:
infer_dataloader=create_dataloaders(32, Data_names, Data_names_key, Labels, is_train=0, batch_size = args.batch_size)

正在读取：normal_0
ndarry 形状： (243938, 1)
tensor 形状： torch.Size([238, 1, 32, 32])
正在读取：IR007_0
ndarry 形状： (121535, 1)
tensor 形状： torch.Size([118, 1, 32, 32])
正在读取：B007_0
ndarry 形状： (121168, 1)
tensor 形状： torch.Size([118, 1, 32, 32])
正在读取：OR007@6_0
ndarry 形状： (121168, 1)
tensor 形状： torch.Size([118, 1, 32, 32])
模型输入的 tensor 形状为： torch.Size([592, 1, 32, 32])
对应标签的 tensor 形状为： torch.Size([592])


In [13]:
infer_dataloader.dataset